In [30]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

In [ ]:
출처 : https://ratsgo.github.io/machine%20learning/2017/04/08/apriori/

장바구니 분석(Market Basket Analysis)

X 아이템을 구매하는 고객들은 Y아이템을 구매할 가능성이 높다.
상품간의 연관성 -> contents-based recommendation의 기본이되는 방법론

In [25]:
items = [["달걀", "라면", "참치캔"], ["라면", "햇반"], ["라면", "콜라"],
         ["달걀", "라면", "햇반"], ["달걀", "콜라"], ["라면", "콜라"], ["라면", "햇반"],
         ["달걀", "라면", "콜라", "참치캔"], ["달걀", "라면", "콜라"], ["양파"]]

len(items)

10

In [26]:
df = pd.DataFrame(index = range(1, 11),
                  columns = ["ID", "Items"])

In [27]:
for idx, item in enumerate(items) :
    df.loc[idx + 1, "ID"] = idx + 1
    df.loc[idx + 1, "Items"] = item

In [38]:
df.loc[1, "Items"]

['달걀', '라면', '참치캔']

In [42]:
df_new = pd.DataFrame(index = range(1, 11),
                      columns = ["달걀", "라면", "참치캔", "햇반", "콜라", "양파"])

In [46]:
for idx in range(1, 11) :
    for col in df_new.columns :   
        if col in df.loc[idx, "Items"] :
            df_new.loc[idx, col] = 1
            
        else :
            df_new.loc[idx, col] = 0

In [47]:
df_new
# 이러한 형태의 sparse matrix가 된다..

,달걀,라면,참치캔,햇반,콜라,양파
1,1,1,1,0,0,0
2,0,1,0,1,0,0
3,0,1,0,0,1,0
4,1,1,0,1,0,0
5,1,0,0,0,1,0
6,0,1,0,0,1,0
7,0,1,0,1,0,0
8,1,1,1,0,1,0
9,1,1,0,0,1,0
10,0,0,0,0,0,1


In [ ]:
규칙
조건절(Antecedent) : 만일 ~라면...
결과절(consequent) : 그 뒷부분..
아이템 집합(Item set) : 조건절 or 결과절을 구성하는 아이템들의 집합
    
ex) 달걀을 구매하는 사람들은 라면도 함께 산다.
조건절 : 달걀 구매
결과절 : 라면 구매
아이템 집합 : "달걀"(조건절), "라면"(결과절)

In [ ]:
빈발 아이템 집합을 판별하는데 사용하는 지지도
조건절A가 일어날 확률.

$$For\quad the\quad rule\quad A\rightarrow B,\\ support(A)=P(A)$$

In [ ]:
조건절 아이템 집합과 결과절 아이템 집합은 상호배반(mutually exclusive)해야한다.

아이템 집합 간 연관성 강도를 측정하는데 쓰는 신뢰도는 조건절(A)가 주어졌을때 결과절이 일어날 조건부 확률

$$confidence(A\rightarrow B)=\frac { P(A,B) }{ P(A) }$$

In [ ]:
생선된 규칙이 실제 효용가치가 있는지 판별하는데 사용되는 향상도
조건절과 결과절이 서로 독립일때와 비교해 두 사건이 동시에 얼마나 발생하는지 비율
향상도가 1이라면 조건절과 결과절은 서로 독립. -> 규칙 사이에 유의미한 연관성이 없다
향상도가 2라면 -> 두사건이 독립일 경우 대비 2배로 긍정적 연관관계

$$lift(A\rightarrow B)=\frac { P(A,B) }{ P(A)\cdot P(B) }$$

In [ ]:
규칙의 효용성은 지지도(support), 신뢰도(confidence), 향상도(lift) 세가지 모두 반영..
규칙1이 규칙2보다 효과적이다 -> 세지표가 모두 다 클 경우에만.!

In [ ]:
규칙을 실제로 만들어볼 차례
가능한 모든 경우의 수를 탐색하여 3개의 지표가 높은 규칙을 찾는것이 가장 ideal한 case
하지만 아이템의 숫자가 많아질수록 계산량도 많아짐 n * (n - 1)
그래서 빈발집합(frequent item sets)만을 고려하여 연관 규칙을 생성하는 A priori algorithm 제안

In [ ]:
아이템 집합 {A}의 지지도(P(A))가 0.1이라면.. {A, B}의 지지도는 maximum 0.1
why? A와 B가 동시에 출현할 확률은 A가 단독으로 출현할 확률과 같거나 낮다
{A, B} = {A}, {B}의 초월 집합(superset)

In [ ]:
임의의 item set X의 지지도가 일정 기준을 넘지 못한다면 해당 item set의 superset은 X의 지지도 보다 낮을 것이고
그러면 유용한 규칙으로 인정 받을 수 없다. -> 해당 집합의 규칙들은 계산할 필요가 없다 -> 계산량 감소

<img src = "./itemset.png">

In [ ]:
{A, B}의 지지도가 최소 지지도 요건을 충족시키지 못할 경우 {A, B} 포함 {A, B}의 superset들도 지지도 요건을
충족시키지 못하므로 해당 superset들의 규칙은 고려할 필요가 없다.

In [49]:
df_new

,달걀,라면,참치캔,햇반,콜라,양파
1,1,1,1,0,0,0
2,0,1,0,1,0,0
3,0,1,0,0,1,0
4,1,1,0,1,0,0
5,1,0,0,0,1,0
6,0,1,0,0,1,0
7,0,1,0,1,0,0
8,1,1,1,0,1,0
9,1,1,0,0,1,0
10,0,0,0,0,0,1


In [ ]:
최소 지지도 요건 = 0.2

P(달걀) = 0.5
P(라면) = 0.8
P(참치캔) = 0.2
P(햇반) = 0.3
P(콜라) = 0.5
-----------------------------------
P(양파) = 0.1

In [ ]:
양파는 최소 지지도 요건을 만족하지 못하므로 양파가 포함된 superset들은 고려할 필요가 없다.

In [ ]:
P(라면,달걀) = 0.4 이런식으로 2개짜리 아이템 집합을 생성하여 표로 나타나면..

In [50]:
pd.DataFrame(data = [[0, 0.4, 0.4, 0.2, 0.2],
                     [0.4, 0, 0.3, 0, 0.2],
                     [0.4, 0.3, 0, 0, 0.1],
                     [0.2, 0, 0, 0, 0],
                     [0.2, 0.2, 0.1, 0, 0]],
            index = ["라면", "달걀", "콜라", "햇반", "참치캔"],
            columns = ["라면", "달걀", "콜라", "햇반", "참치캔"])

,라면,달걀,콜라,햇반,참치캔
라면,0.0,0.4,0.4,0.2,0.2
달걀,0.4,0.0,0.3,0.0,0.2
콜라,0.4,0.3,0.0,0.0,0.1
햇반,0.2,0.0,0.0,0.0,0.0
참치캔,0.2,0.2,0.1,0.0,0.0


In [ ]:
위와같은 대각행렬의 형태. 여기에서도 최소 지지도 0.2를 채우지 못한 case들은 고려하지 않는다.

In [ ]:
이런식으로 최소 지지도 이상을 나타내는 아이템 집합이 없을때까지 item set의 크기를 1씩 증가시키며 반복수행한다.
이렇게 구한 상위 연관규칙 3가지는

In [51]:
pd.DataFrame(data = [
                    ["참치캔", "달걀, 라면", 2, 1, 2.5],
                    ["참치캔", "달걀", 2, 1, 2],
                    ["라면, 참치캔", "달걀", 2, 1, 2]
                    ],
             columns = ["조건절", "결과절", "지지도", "신뢰도", "향상도"])

,조건절,결과절,지지도,신뢰도,향상도
0,참치캔,"달걀, 라면",2,1,2.5
1,참치캔,달걀,2,1,2.0
2,"라면, 참치캔",달걀,2,1,2.0


In [ ]:
문서요약에도 적용 가능. 한문장을 트랜잭션 데이터로 상정, 문장내 출현한 단어를 아이템으로 놓고 같은 방식 적용
여기서의 유의미한 규칙은 co-occurence 정보를 함축했을 것
ex) 조건절 : (발, 없는, 말이) -> 결과절 (간다)